In [1]:
import os
import torch
import torchvision.models as models

### Model architecute print

In [2]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [12]:
from torchsummary import summary

model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [30]:
import torch
import torchvision.transforms as T
import torchvision.models as models

from PIL import Image


def hook_fn(module, input, output):    
    print(f"[HOOK] Layer: {module.__class__.__name__}")

    # print conv layer info
    if isinstance(module, torch.nn.Conv2d):
        print("  Conv2d Info:")
        print("    in_channels:", module.in_channels)
        print("    out_channels:", module.out_channels)
        print("    kernel_size:", module.kernel_size)
        print("    stride:", module.stride)
        print("    padding:", module.padding)
        print("    dilation:", module.dilation)
        print("    groups:", module.groups)

        # ---- Weight 출력 ----
        if hasattr(module, "weight") and module.weight is not None:
            w = module.weight.detach().cpu()

            if isinstance(module, torch.nn.Conv2d):
                # Conv layer weight: (C_out, C_in, kH, kW) -> [C_in * K * K, C_out]
                w_reshaped = w.permute(1, 2, 3, 0).reshape(-1, w.shape[0])
                print("  Weight shape (reshaped):", tuple(w_reshaped.shape))
            else:
                print("  Weight shape:", tuple(w.shape))
                w_reshaped = w
            
            print("  Weight first 10 elements:",
                w_reshaped.flatten()[:10].tolist())
        else:
            print("  (No weight)")

        # ---- Bias 출력 ----
        if hasattr(module, "bias") and module.bias is not None:
            b = module.bias.detach().cpu()
            print("  Bias shape:", tuple(b.shape))
            print("  Bias first 10 elements:",
                b.flatten()[:10].tolist())
        
        x = input[0].detach().cpu()  # (1,C,H,W)
        x_hwc = x.squeeze(0).permute(1,2,0).contiguous()
        print("Input shape (HWC):", x_hwc.shape)
        print("Input first 10:", x_hwc.flatten()[:10].tolist())
        
        # convert NCHW to HWC and show first 10 elements
        if output.dim() == 4:  # Conv layer output
            output_hwc = output.squeeze(0).permute(1, 2, 0)
        else:
            output_hwc = output.squeeze(0)

        print("Output shape (HWC):", output_hwc.shape)
        print("First 10 elements of output tensor:")
        print(output_hwc.flatten()[:10].tolist())
        print("-" * 50)


debug = True
img_path = "../img/shark.png"

# 1) 모델 로드
weights = models.MobileNet_V2_Weights.IMAGENET1K_V1
model = models.mobilenet_v2(weights=weights)
model.eval()

# Forward Hook 등록
if debug:
    for name, module in model.named_modules():
        module.register_forward_hook(hook_fn)

# 2) Resize 224×224 + Normalize(ImageNet)
preprocess = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# 3) 이미지 로드 & 전처리
img = Image.open(img_path).convert("RGB")
input_tensor = preprocess(img).unsqueeze(0)  # (1,3,224,224)

input_tensor_HWC = input_tensor.squeeze(0).permute(1, 2, 0)
print("Input Tensor Shape (HWC):", input_tensor_HWC.shape)
print("First 10 elements of Input Tensor:"
      , input_tensor_HWC.flatten()[:10].tolist())
print("-" * 50)


# 4) 추론
with torch.no_grad():
    logits = model(input_tensor)



Input Tensor Shape (HWC): torch.Size([224, 224, 3])
First 10 elements of Input Tensor: [-1.158917784690857, -0.14495795965194702, 0.8273640275001526, -1.158917784690857, -0.17997196316719055, 0.8970807790756226, -1.1246683597564697, -0.12745095789432526, 1.0888018608093262, -1.1931673288345337]
--------------------------------------------------
[HOOK] Layer: Conv2d
  Conv2d Info:
    in_channels: 3
    out_channels: 32
    kernel_size: (3, 3)
    stride: (2, 2)
    padding: (1, 1)
    dilation: (1, 1)
    groups: 1
  Weight shape (reshaped): (27, 32)
  Weight first 10 elements: [0.013185339979827404, -0.06369444727897644, -0.10511069744825363, 0.002303319750353694, -0.06910442560911179, 0.15895509719848633, -0.01961679197847843, 0.02333073690533638, 0.052242208272218704, 7.297255069715902e-05]
Input shape (HWC): torch.Size([224, 224, 3])
Input first 10: [-1.158917784690857, -0.14495795965194702, 0.8273640275001526, -1.158917784690857, -0.17997196316719055, 0.8970807790756226, -1.124668